# 1 Single Capital Model

This notebook describes the code used to solve the model described in Section 4.4. That is, we first abstract away from model ambiguity and restrict to a single capital stock. Suppose that the logarithm of continuation utility $V$ evolves as:

$$
d {\widehat V}_t = {\hat \mu}_{v,t} dt + \sigma_{v,t} \cdot dB_t
$$

where $ {\hat \mu}_{v,t}$ is the local mean, $ |\sigma_{v,t}|^2$ is local variance and $dB_t$ is a Brownian increment, such that:

$$
0  =   \left({\frac \delta {1 - \rho}} \right) \left[ (C_t/V_t)^{1 - \rho}
  - 1  \right]  + {\hat \mu}_{v,t}  +  \left(\frac{1 - \gamma}{ 2 } \right)  \vert \sigma_{v,t} \vert^2 
$$

We presume that the stochastic component of technology growth $Z_1$ and aggregate stochastic volatility $Z_2$ are governed by the following processes:

$$
d Z_{t}^1 =  - \beta_{1} Z_t^1 dt +  \sqrt{Z_t^2} \sigma_1 \cdot d B_{t} \\
d Z_t^2 = - \beta_2 ( Z_t^2 - \mu_2) dt +  \sqrt{Z_t^2} \sigma_2 \cdot d B_t
$$

Capital growth is governed by:

$$
dK_t = K_t\left[ \Phi \left( \frac{I_t}{K_t} \right) + \beta_k Z_t^1 - \eta_k \right] dt  + K_t \sqrt{Z_t^2}  {\sigma}_k \cdot dB_t
$$

Where:

$$
\Phi(i) = {\frac 1 \phi}  \log \left( 1 + \phi i \right)
$$

Output is constrained by:

$$
C_t + I_t = \alpha K_t
$$

From this we can deduce the Hamilton-Jacobi-Bellman (HJB) equation:

$$
0 =   \max_{c+i = \alpha}  \left\{ \left( \frac{\delta}{1 - \rho} \right) \left( c^{1-\rho} \exp \left[ (\rho - 1) \upsilon \right] - 1 \right) + \Phi(i) + \beta_k z_1 - \eta_k - \frac{1}{2} z_2 |\sigma_k|^2 \right. \\
 \left. + \mu_z \cdot \frac {\partial \upsilon}{\partial z}  +  \frac {z_2}2  {\rm trace} \left\{ {\sigma_z}' \frac {\partial^2 \upsilon}{\partial z \partial z'} \sigma_z \right\} +  \frac{(1 - \gamma) z_2 }{2} \left\vert\sigma_k + {\sigma_z}' \frac {\partial \upsilon}{\partial z}   \right\vert^2 \right\}
$$

Where $c$ is the consumption-to-capital ratio and $i$ is the investment-to-capital ratio. We solve this HJB using the algorithm described in Supplemental Appendix C.1.

<!-- | **Parameter**  | **Value**                                                         |
|----------------|-------------------------------------------------------------------|
| $\eta_k$       | 0.04                                                              |
| $\phi$         | 8                                                                 |
| $\beta_k$      | 0.04                                                              |
| $\beta_1$      | 0.056                                                             |
| $\beta_2$      | 0.194                                                             |
| $\mu_2$        | $6.3 \times 10^{-6}$                                              |
| **Upper triangular**  
| $\sigma_2$ |$\sqrt{12} \times [0.92, 0.40, 0]$                              |
| **Lower triangular** | $\sqrt{12} \times [1, 0, 0]$                                    |
| $\sigma_k$     | $\sqrt{12} \times [0.92, 0.40, 0]$                                |
| $\sigma_1$     | $\sqrt{12} \times [0, 5.7, 0]$                                    |
| **Lower triangular** | $\sqrt{12} \times [2.3, 5.2, 0]$                                |
| $\sigma_2$     | $\sqrt{12} \times [0, 0, 0.00031]$                                | -->

## 1.2 Solution Overview

The function `main_one_capital` solves the above HJB equation. It uses functions from `utils_onecapital`, which first-time users should not change. `main_onecapital` takes in the following inputs:

```{list-table}
:header-rows: 1

* - Input
  - Notation in paper
  - Description
* - `Delta`
  - $\Delta_t$
  - False-transient step-size: lower values (0.01 or 0.001) reduce likelihood of convergence issues
* - `gamma`
  - $\gamma$
  - Uncertainty aversion
* - `delta`
  - $\delta$
  - Discount rate
* - `rho`
  - $\rho$
  - Inverse of IES
* - `alpha`
  - $\alpha$
  - Productivity
```

The user can also edit the `main_onecapital` Julia file to change model parameters which are held fixed in the paper, including:

```{list-table}
:header-rows: 1

* - Input
  - Notation in paper
  - Description
* - `cons`
  - $c$
  - Consumption-output ratio
* - `a11`
  - $\beta_1$
  - Persistence of $Z_1$
* - `a22`
  - $\beta_2$
  - Persistence of $Z_2$
* - `ymean`
  - $\mu_2$
  - Mean of $Z_2$
* - `phi`
  - $\phi$
  - Adjustment cost
* - `sigma_k`
  - $\sigma_k$
  - Shock exposure for $dK$
* - `sigma_z`
  - $\sigma_1$
  - Shock exposure for $dZ_1$
* - `sigma_y`
  - $\sigma_2$
  - Shock exposure for $dZ_2$
```

Finally, the following parameters govern the construction of the state space grid and convergence criteria:
```{list-table}
:header-rows: 1

* - Input
  - Description
* - `zmin`,`zmax`
  - Minimum and maximum values of $Z_1$ grid; an insufficiently large grid may cause inaccuracies
* - `II`
  - Number of $Z_1$ gridpoints
* - `ymin`,`ymax`
  - Minimum and maximum values of $Z_2$ grid; an insufficiently large grid may cause inaccuracies
* - `JJ`
  - Number of $Z_2$ gridpoints
* - `maxit`
  - Maximum number of value function iterations 
* - `crit`
  - The algorithm stops when the difference between $V_t$ and $V_{t+1}$ is less than `crit` for all points on the state-space grid
```

`main_onecapital` outputs a dictionary that contains the above parameters, as well as the following outputs:
```{list-table}
:header-rows: 1

* - Output
  - Notation in paper
  - Description
* - `V`
  - $V$
  - Value function
* - `c`
  - $\frac{C}{K}$
  - Consumption-capital ratio
* - `d`
  - $\frac{I}{K}$
  - Investment-capital ratio
* - `Vz`
  - $\frac{\partial{v}}{\partial{z_1}}$
  - Derivative of $V$ with respect to $Z^1$
* - `Vy`
  - $\frac{\partial{v}}{\partial{z_2}}$
  - Derivative of $V$ with respect to $Z^2$
* - `mu_k`
  - $\mu_{\hat{K}}$
  - Drift of $\log{K}$
* - `g`
  - 
  - Stationary density
* - [`hk`,`hz`,`hy`]
  - $\sigma_k + \sigma_z' \frac{\partial{v}}{\partial{z}}$
  - Robust control term
* - `ymean`
  - $\mu_2$
  - Mean of $Z_2$
* - `phi`
  - $\phi$
  - Adjustment cost
* - `sigma_k`
  - $\sigma_k$
  - Shock exposure for $dK$
* - `sigma_z`
  - $\sigma_1$
  - Shock exposure for $dZ_1$
* - `sigma_y`
  - $\sigma_2$
  - Shock exposure for $dZ_2$

## 1.3 Quick Start
We begin by importing required libraries:

In [1]:
using LinearAlgebra
using SparseArrays
using Interpolations
using SuiteSparse
using Plots
include("src/1/utils_onecapital.jl")
include("src/1/main_onecapital.jl")

  Activating project at `/project/lhansen/macrofinance`


Next we set parameters, including an `action_name` which determines where the file is saved:

In [2]:
alpha = 0.0922
delta = 0.01
gamma = 1.0
Delta = 1.0
rho = 1.0
action_name = "test"

"test"

Now we can run our main function:

In [3]:
results = main(gamma, rho, Delta, delta, alpha, action_name)

zmax = 5.0, zmin = -5.0, zlength = 201
ymax = 2.0e-5, ymin = 5.0e-7, ylength = 201
----------------------------------
Iteration = 1
Distance = 0.2253588241838127
v max = -5.774641175816187
v min = -6.149840658685116
----------------------------------
----------------------------------
Iteration = 2
Distance = 0.21336164298098925
v max = -5.561279532835198
v min = -6.28844158588914
----------------------------------
----------------------------------
Iteration = 3
Distance = 0.2020913901009722
v max = -5.359188142734226
v min = -6.416520157178582
----------------------------------
----------------------------------
Iteration = 4
Distance = 0.19150073759421993
v max = -5.167687405140006
v min = -6.534747094790304
----------------------------------
----------------------------------
Iteration = 5
Distance = 0.1815472483518974
v max = -4.9861401567881085
v min = -6.643751090716898
----------------------------------
----------------------------------
Iteration = 6
Distance = 0.1721915014381

Dict{String, Any} with 40 entries:
  "ymax"    => 2.0e-5
  "V"       => [-5.21985 -5.21984 … -5.213 -5.213; -5.1899 -5.1899 … -5.18883 -…
  "dy"      => 9.75e-8
  "a22"     => 0.194
  "zz"      => [-5.0 -5.0 … -5.0 -5.0; -4.95 -4.95 … -4.95 -4.95; … ; 4.95 4.95…
  "times"   => 94.5006
  "hz"      => [7.30085 7.29732 … 6.15858 6.15821; 13.2831 13.2795 … 11.9034 11…
  "delta"   => 0.01
  "mu_y"    => [1.1252e-6 1.10629e-6 … -2.63889e-6 -2.6578e-6; 1.1252e-6 1.1062…
  "hy"      => [0.097075 0.241849 … 0.0415655 0.014188; -0.00158519 -0.00420511…
  "sigma_y" => [0.0, 0.0, 0.00107387]
  "phi"     => 8.0
  "zmin"    => -5.0
  "maxit"   => 50000
  "Delta"   => 1.0
  "crit"    => 1.0e-5
  "d"       => [0.0761111 0.0761111 … 0.0761111 0.0761111; 0.0761111 0.0761111 …
  "zmax"    => 5.0
  "sigma_k" => [3.18697, 1.38564, 0.0]
  "dz"      => 0.05
  "g"       => [3.00353e-8 2.13511e-8 … 5.2324e-17 5.08546e-17; 1.44656e-8 1.42…
  "hk"      => [3.18697 3.18697 … 3.18697 3.18697; 3.18697 3.18697 … 3.1

By default, the solution is saved to the `output` folder. These results can be used to compute elasticities, which are described in a different section.